# 鸭子类型
得益于python的数据模型，自定义类型的行为可以像内置类型一样自然。实现这一点靠的不是继承，而是鸭子类型：只需要按照预定行为（协议）实现对象所需的方法即可。这一点其实相当于python的自定义类默认会含有很多常用协议的接口，可以实现来变成各种各样行为的“鸭子” 

本章探讨的常见行为如下：  
1.获取对象常见表示形式的内置函数  
2.使用类方法实现备选构造方法  
3.扩展内置的format和str.format所使用的格式微语言  
4.实现只读属性  
5.实现对象的散列  
6.利用\__slot__节省内存
# 对象表示形式
## repr()
返回便于开发者理解的对象的字符串表示形式，如：创建对象的语句
## str()
返回便于用户理解的对象字符串表达方式
## bytes()
返回对象的字节序列表示，会调用\__bytes__

In [48]:
v1 = Vector2d(3, 4)
x, y = v1
print(x, y)
print(v1)
print(repr(v1))
v1_clone = eval(repr(v1))
print(v1 == v1_clone)
# 这里打印17个字节（可ascii解码的直接显示）
print(bytes(v1))

3.0 4.0
(3.0, 4.0)
Vector2d(3.0, 4.0)
True
b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'


## format()/str.format()
返回对象使用特殊格式代码的字符串表示，会调用\__format__(format_spec)，format_spec为格式说明符。

In [45]:
brl = 1/2.43
print(format(brl, '0.4f'))

import collections
Eco = collections.namedtuple('Eco', 'name rate')
brl = Eco(name = 'BRL', rate = brl)

# {0.rate:0.2f}是代换用法':'前表示代换字段（可为空），0.rate表示使用第0个参数的rate属性，0.2f为格式说明符，会调用0.rate的__format__('0.2f')
print('1 {0.name} = {0.rate:0.2f} USD'.format(brl))

0.4115
1 BRL = 0.41 USD


格式说明符会被传入自定义的\__format__因此可以自己定义格式规范。如果没有定义\__format__则会调用从object继承的方法，该方法直接返回str(self),如果试图传入格式说明符则会报错。

In [44]:
from datetime import datetime
now = datetime.now()

"It's now {:%I:%M %p}".format(now)

"It's now 08:13 PM"

# classmethod和staticmethod
## classmethod
classmethod用于定义操作类的方法，第一个参数是类本身而不是示例，因此classmethod常用于定义备选的构造方法。
## staticmethod
staticmethod静态方法装饰的函数和普通函数一样，只不过该方法存在于类的定义体中而非在模块层定义，用于将仅与该类相关的静态方法放在类定义体中。
# 实现可散列
坐标依赖于x,y坐标，一个Vector2d实例若可散列则散列值绝不可变化，且实例必须不可变（至少散列值依赖的属性不可变）因此需要将x,y属性设为私有属性，同时类中使用@property装饰器将方法标记为只读属性，通过属性的方式读取值。同时hash值使用x,y的散列值，并通过按位异或来组合。
# 私有属性和受保护属性
python不像Java有private修饰符用于创建私有属性，但python有相应的机制避免子类意外覆盖父类的‘私有’属性。python使用双下划线的属性会变成
'\_类名\_\_属性名'。这种名称改写可使得父类和子类的同名‘私有’属性被各自保留。当然子类中也是有'\_父类类名\_\_属性名’这个属性的。

In [55]:
v1 = Vector2d(3, 4)
# python会把属性名存入实例的__dict__属性中
v1.__dict__

{'_Vector2d__x': 3.0, '_Vector2d__y': 4.0}

# 使用\__slots__类属性节省空间
python的实例中以\__dict__存储实例属性，但是dict由于使用了散列表，由于散列表使用的是一个稀疏数组（bucket），因此空间利用率低，如果处理数百万个属性不算多的实例，内存开销很大，因此可利用\__slots__，其作用是python在处理对应实例时使用类似元组的结构存储实例变量，并取消消耗内存的\__dict__属性（因此也无法使对象增加新的属性）,从而只能使用\__slots__定义的属性名。

如果将'\__dict__'字符串添加到\__slots__使得实例可拥有\__dict__属性，则实例可支持动态属性的创建，实例会使用\__dict__保存新加的属性。另外如果使用了\__slots__后需要实例支持弱引用则需要额外增加‘\__weakref__’字符串。

## \__slots__总结
#### 1.每个子类都要定义\__slots__，因为解释器会忽略继承得到的\__slots__
#### 2.实例只能使用\__slots__定义的属性名，且不支持新增属性，除非把‘\__dict__’加入\__slots__（这样做可能失去了节省内存的初衷）
#### 3.如果希望支持弱引用则需要有‘\__weakref__’

# 覆盖类属性
python的类属性是属于类的，因此当使用实例访问类属性时默认获取了类的属性值，如果使用类实例为该属性赋值实际上是为实例创建了新的同名实例属性，该属性会覆盖其类的属性值。因此类的属性可用于提供默认值。在Java中类的属性同样属于类，但你无法直接为类实例动态添加实例属性。

In [59]:
v2 = Vector2d(3, 4)
print(bytes(v2))
print(len(bytes(v2)))
# 设置为单精度浮点（4B），array.array以单精度浮点数创建数组
v2.typecode = 'f'
print(bytes(v2))
# 8个字节+1个字节的typecode
print(len(bytes(v2)))

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'
17
b'f\x00\x00@@\x00\x00\x80@'
9


## 类属性修改
如果想改变类属性为实例提供新的默认值，常用做法不是直接修改类的属性值，而是通过继承然后修改对应属性，这种方法更加有针对性，相当于新建定制类。

In [63]:
class ShortVector2d(Vector2d):
    typecode = 'f'
    
v3 = ShortVector2d(1/11, 1/27)
print(repr(v3))
print(bytes(v3))
print(len(bytes(v3)))

ShortVector2d(0.09090909090909091, 0.037037037037037035)
b'f\x8c.\xba=&\xb4\x17='
9


# 最终版本
Vector2d‘最全’版本如下，它使用特殊方法支持了非常多的python内部定义的行为，是符合python风格的类，但使用中特殊方法的支持程度需要以实际为依据，__要构建符合python风格的类，就要观察真正python类对象的行为，通过需要的行为去实现对应的方法。__

In [1]:
from array import array
import math

class Vector2d:
    # 类属性，默认使用双精度浮点数
    typecode = 'd'
    
    # 使用‘私有’属性，私有属性会进行名称修改为‘_类名__属性名’
    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)
        
    # 只读特性    
    @property
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y
    
    # 创建迭代器，这使得类实例可迭代，可直接用于拆包。
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    # 返回创建当前实例的字符串表示，这里利用了类实例的可迭代性直接拆包
    def __repr__(self):
        # 使用获取类名的方式来支持子类。
        class_name = type(self).__name__
        # !r表示使用repr的方式进行格式化
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    # 返回便于理解的字符创表示形式
    def __str__(self):
        return str(tuple(self))
    
    # 将实例转换为字节序列（不可变类型），存储对象存储格式和对象的array.array。
    def __bytes__(self):
        # 记录typecode的ascii编码+将对象转为array.array类型后再进行字节化
        # array.array默认以8B双精度浮点数存储每一个元素，因此共16个字节 + 额外标记'd'共17字节
        return (bytes(self.typecode, 'ascii') + bytes(array(self.typecode, self)))
              
    def __eq__(self, other):
        return tuple(self) == tuple(other)
              
    def __hash__(self, other):
        return hash(self.x) ^ hash(self.y)
        
    def __abs__(self):
        return math.hypot(*self)
              
    def __bool__(self):
        return abs(self)
    
    def angle(self):
        return math.atan2(self.y, self.x)
    
    # 实现自定格式规范符，标准格式规范符+使用坐标类型
    def __format__(self, fmt_spec=''):
        # 极坐标展示
        if fmt_spec.endwith('p'):
            # 提取剩余标准规范符
            fmt_spec = fmt_spec[:-1]
            # 定义使用的坐标
            coords = (abs(self), self.angle())
            # 定义最后的展示格式
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        
        # 首先用标准规范符展示每个坐标元素然后用最终格式展示
        components = (format(c, fmt_spec) for c in coords)
        # 使用可迭代元素的形参拆包传参
        return outer_fmt.format(*components)
    
    @classmethod
    def frombytes(cls, octets):
        # 将序列化的对象第一个字节转为字符，得到对象存储的单位标识。
        typecode = chr(octets[0])
        # 以单位标识读取剩余字节对应的内存空间，并使用存储的格式（默认使用的是双精度浮点数，因此每次memoryview以8B读取）对其进行读取。
        memv = memoryview(octets[1:]).cast(typecode)
        # 直接新建对象
        return cls(*memv)